### packages

In [277]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import copy


## data exploration

In [57]:
data_info = pd.read_csv("./metadata.csv")
data_info = data_info.fillna(0)
# data_info

In [58]:
# for sample S0000 - first sample
data_info_train = data_info.loc[data_info.split=='train']
data_info_test = data_info.loc[data_info.split=='val']


In [60]:
data_label = pd.read_csv('./train_labels.csv')

This is the label count of label dataset

In [61]:
print("output".ljust(50) , " 0".ljust(5), "1".ljust(10))
print()
for cols in data_label.columns[1:]:
    print(str(cols).ljust(50) , np.array(data_label[cols].value_counts()))

output                                              0    1         

aromatic                                           [730  79]
hydrocarbon                                        [544 265]
carboxylic_acid                                    [694 115]
nitrogen_bearing_compound                          [713  96]
chlorine_bearing_compound                          [791  18]
sulfur_bearing_compound                            [778  31]
alcohol                                            [783  26]
other_oxygen_bearing_compound                      [788  21]
mineral                                            [692 117]


In [107]:
# plt.plot([1, 2, 3, 4], [1, 4, 9, 16])
# ysdf = 0
cols = list(data_info.columns)
cols.remove('features_md5_hash')
cols
label_names = ['sample_id', 'aromatic', 'hydrocarbon', 'carboxylic_acid',
       'nitrogen_bearing_compound', 'chlorine_bearing_compound',
       'sulfur_bearing_compound', 'alcohol', 'other_oxygen_bearing_compound',
       'mineral']

In [114]:
len(data_info)

1121

## feature extraction from dataset
taking segments of dataset and appling mean to them

In [118]:

feature_length = 300 ## extracting these many values from a data set entry

X = []
y = []
test = []

for i in range(0,len(data_info)):
    sample_id, split, derivatized, path = data_info[cols].iloc[i]
    if split == 'train':
        label = data_label.iloc[i][1:]
    data = pd.read_csv(path)

    df = data
    data_length = len(df)
    batch_size = data_length // feature_length

    # Initialize lists to store the batch features
    mean_time = []
    mean_mass = []
    mean_intensity = []

    features = []
    # Iterate over the batches
    for i in range(feature_length):
        # Calculate the start and end indices of the current batch
        start_idx = i * batch_size
        end_idx = start_idx + batch_size

        # Extract the current batch from the dataset
        batch = df.iloc[start_idx:end_idx]

        # Calculate the mean of time, mass, and intensity for the current batch
        mean_time.append(batch['time'].mean())
        mean_mass.append(batch['mass'].mean())
        mean_intensity.append(batch['intensity'].mean())
    features = mean_mass + mean_time + mean_intensity
    features.append(derivatized)
    if split == 'train':
        X.append(copy.deepcopy(features))
        y.append(label)
    else:
        test.append(copy.deepcopy(features))

    #for data visualization

    # Combine the batch features into a single dataframe
    # batch_features = pd.DataFrame({
    #     'mean_time': mean_time,
    #     'mean_mass': mean_mass,
    #     'mean_intensity': mean_intensity
    # })


    # plt.scatter(batch_features['mean_mass'] ,batch_features['mean_intensity'],s = 2 )
    # # plt.xlim(1,70)
    # plt.ylim(1,2e7)
    # plt.show()
    # break


In [204]:

# sns.lineplot(data = df, x= 'time', y = 'intensity')
X_np = np.array(copy.deepcopy(X)).astype(float)
y_np = np.array(copy.deepcopy(y)).astype(float)
test_np = np.array(copy.deepcopy(test)).astype(float)


In [278]:
print(f"shape of X : {X_np.shape}\n"+
      f"shape of y : {y_np.shape}\n"+
      f"shape of test : {test_np.shape}\n")


shape of X : (809, 901)
shape of y : (809, 9)
shape of test : (312, 901)



In [206]:
df_X = pd.DataFrame(X_np)
df_y = pd.DataFrame(y_np)
df_test = pd.DataFrame(test_np)

#saving to csv
df_X.to_csv("df_X.csv")
df_y.to_csv("df_y.csv")
df_test.to_csv("df_test.csv")



nan data 
-- no nan rows or columns

In [207]:
nan_cols_x = [col for col in df_test if df_X[col].isna().any()]   
nan_cols_test = [col for col in df_test if df_X[col].isna().any()]   

# Preprocessing and Testing
standand scaler
minmax scaler


In [290]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import math

### LogisticRegression model

In [279]:
classes = list(data_label.columns[1:] )

In [281]:
# unscaled data using logistic regression
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.33, random_state=42)
for i,cols in enumerate(classes):
    model = LogisticRegression(max_iter = 1000000)
    model.fit(X_train, y_train[:,i])
    preds = model.predict_proba(X_test)
    pos = int(100*y_np[:,i].sum()/809)
    print(cols.ljust(30," "),
        "roc - ",math.ceil(roc_auc_score(y_test[:,i], preds[:, 1])*100),
        "   mse - ",math.ceil(mean_squared_error(y_test[:,i], preds[:, 1])*100),
        "   positive percentage - ", pos)

# results are variable for different types
# for random_state in [10 ,239,3,94023,234,342,234]:
#     X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, test_size=0.33, random_state=random_state)
#     for i,cols in enumerate(classes):
#         if (i==0):
#             continue
#         model = LogisticRegression(max_iter = 1000000)
#         model.fit(X_train, y_train[:,i])
#         preds = model.predict_proba(X_test)
#         print(str(random_state).ljust(10," "),cols.ljust(50," "),roc_auc_score(y_test[:,i], preds[:, 1]))
#         break


aromatic                       roc -  65    mse -  17    positive percentage -  9
hydrocarbon                    roc -  73    mse -  27    positive percentage -  32
carboxylic_acid                roc -  61    mse -  27    positive percentage -  14
nitrogen_bearing_compound      roc -  71    mse -  17    positive percentage -  11
chlorine_bearing_compound      roc -  48    mse -  7    positive percentage -  2
sulfur_bearing_compound        roc -  60    mse -  10    positive percentage -  3
alcohol                        roc -  55    mse -  10    positive percentage -  3
other_oxygen_bearing_compound  roc -  67    mse -  6    positive percentage -  2
mineral                        roc -  72    mse -  17    positive percentage -  14


In [282]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_np_minmax = scaler.fit_transform(X_np) 
test_np_minmax = scaler.transform(test_np) 

In [283]:
#min max scaled input
X_train, X_test, y_train, y_test = train_test_split(X_np_minmax, y_np, test_size=0.33, random_state=42)
for i,cols in enumerate(classes):
    model = LogisticRegression(max_iter = 1000000)
    model.fit(X_train, y_train[:,i])
    preds = model.predict_proba(X_test)
    pos = int(100*y_np[:,i].sum()/809)
    print(cols.ljust(30," "),
        "roc - ",math.ceil(roc_auc_score(y_test[:,i], preds[:, 1])*100),
        "   mse - ",math.ceil(mean_squared_error(y_test[:,i], preds[:, 1])*100),
        "   positive percentage - ", pos)


aromatic                       roc -  86    mse -  8    positive percentage -  9
hydrocarbon                    roc -  87    mse -  14    positive percentage -  32
carboxylic_acid                roc -  94    mse -  9    positive percentage -  14
nitrogen_bearing_compound      roc -  93    mse -  8    positive percentage -  11
chlorine_bearing_compound      roc -  98    mse -  3    positive percentage -  2
sulfur_bearing_compound        roc -  90    mse -  3    positive percentage -  3
alcohol                        roc -  93    mse -  3    positive percentage -  3
other_oxygen_bearing_compound  roc -  90    mse -  3    positive percentage -  2
mineral                        roc -  81    mse -  9    positive percentage -  14


In [284]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_np_std = std_scaler.fit_transform(X_np) 
test_np_std = std_scaler.transform(test_np) 

In [285]:
# standard scaled input
X_train, X_test, y_train, y_test = train_test_split(X_np_std, y_np, test_size=0.33, random_state=42)
for i,cols in enumerate(classes):
    model = LogisticRegression(max_iter = 1000000)
    model.fit(X_train, y_train[:,i])
    preds = model.predict_proba(X_test)
    pos = int(100*y_np[:,i].sum()/809)
    print(cols.ljust(30," "),
        "roc - ",math.ceil(roc_auc_score(y_test[:,i], preds[:, 1])*100),
        "   mse - ",math.ceil(mean_squared_error(y_test[:,i], preds[:, 1])*100),
        "   positive percentage - ", pos)

aromatic                       roc -  75    mse -  9    positive percentage -  9
hydrocarbon                    roc -  83    mse -  18    positive percentage -  32
carboxylic_acid                roc -  91    mse -  11    positive percentage -  14
nitrogen_bearing_compound      roc -  91    mse -  10    positive percentage -  11
chlorine_bearing_compound      roc -  98    mse -  3    positive percentage -  2
sulfur_bearing_compound        roc -  83    mse -  3    positive percentage -  3
alcohol                        roc -  92    mse -  4    positive percentage -  3
other_oxygen_bearing_compound  roc -  86    mse -  4    positive percentage -  2
mineral                        roc -  76    mse -  12    positive percentage -  14


In [286]:
ts = """               precision    recall  f1-score   support

         0.0       0.90      1.00      0.95       239
         1.0       """
len(ts)

129

In [287]:
# for low positive compunds the algorithm performs worst and result is highly variable with random state
X_train, X_test, y_train, y_test = train_test_split(X_np_minmax, y_np, test_size=0.33, random_state=43)
for i,cols in enumerate(classes):
    model = LogisticRegression(max_iter = 1000000)
    model.fit(X_train, y_train[:,i])
    preds = model.predict(X_test)
    pos = int(100*y_np[:,i].sum()/809)
    # print(cols,'\n', classification_report(y_test[:,i], preds))
    print(cols.ljust(30," "),'positive accuarcy', classification_report(y_test[:,i], preds,zero_division=True)[len(ts)-1:len(ts)+3])

aromatic                       positive accuarcy 0.40
hydrocarbon                    positive accuarcy 0.77
carboxylic_acid                positive accuarcy 0.53
nitrogen_bearing_compound      positive accuarcy 0.44
chlorine_bearing_compound      positive accuarcy 0.00
sulfur_bearing_compound        positive accuarcy 0.50
alcohol                        positive accuarcy 1.00
other_oxygen_bearing_compound  positive accuarcy 1.00
mineral                        positive accuarcy 0.79


In [289]:
# result highly depends on random_state
sel_cols = "aromatic"
for rs in [231,3123,12,3,123,32,3,13,1000000,1010010,1232,32313,45362,562456]:
    X_train, X_test, y_train, y_test = train_test_split(X_np_minmax, y_np, test_size=0.33, random_state=rs)
    for i,cols in enumerate(classes):
        if (cols!=sel_cols):
            continue
        model = LogisticRegression(max_iter = 1000000)
        model.fit(X_train, y_train[:,i])
        preds = model.predict(X_test)
        pos = int(100*y_np[:,i].sum()/809)
        # print(cols,'\n', classification_report(y_test[:,i], preds))
        print(cols.ljust(30," "),'positive accuarcy', classification_report(y_test[:,i], preds,zero_division=True)[len(ts)-1:len(ts)+3],"      total positive ", pos)
        break

aromatic                       positive accuarcy 0.38       total positive  9
aromatic                       positive accuarcy 0.62       total positive  9
aromatic                       positive accuarcy 0.30       total positive  9
aromatic                       positive accuarcy 0.80       total positive  9
aromatic                       positive accuarcy 0.67       total positive  9
aromatic                       positive accuarcy 0.40       total positive  9
aromatic                       positive accuarcy 0.80       total positive  9
aromatic                       positive accuarcy 0.33       total positive  9
aromatic                       positive accuarcy 0.50       total positive  9
aromatic                       positive accuarcy 0.44       total positive  9
aromatic                       positive accuarcy 0.44       total positive  9
aromatic                       positive accuarcy 0.62       total positive  9
aromatic                       positive accuarcy 0.75       tota

In [292]:
# gausian nb
sel_cols = "aromatic"
for rs in [231,3123,12,3,123,32,3,13,1000000,1010010,1232,32313,45362,562456]:
    X_train, X_test, y_train, y_test = train_test_split(X_np_minmax, y_np, test_size=0.33, random_state=rs)
    for i,cols in enumerate(classes):
        if (cols!=sel_cols):
            continue
        model = GaussianNB()
        model.fit(X_train, y_train[:,i])
        preds = model.predict(X_test)
        pos = int(100*y_np[:,i].sum()/809)
        # print(cols,'\n', classification_report(y_test[:,i], preds))
        print(cols.ljust(30," "),'positive accuarcy', classification_report(y_test[:,i], preds,zero_division=True)[len(ts)-1:len(ts)+3],"      total positive ", pos)
        break

aromatic                       positive accuarcy 0.25       total positive  9
aromatic                       positive accuarcy 0.26       total positive  9
aromatic                       positive accuarcy 0.33       total positive  9
aromatic                       positive accuarcy 0.31       total positive  9
aromatic                       positive accuarcy 0.23       total positive  9
aromatic                       positive accuarcy 0.29       total positive  9
aromatic                       positive accuarcy 0.31       total positive  9
aromatic                       positive accuarcy 0.22       total positive  9
aromatic                       positive accuarcy 0.25       total positive  9
aromatic                       positive accuarcy 0.21       total positive  9
aromatic                       positive accuarcy 0.20       total positive  9
aromatic                       positive accuarcy 0.27       total positive  9
aromatic                       positive accuarcy 0.23       tota

Since gausianNB depends on probability accuracy is worst,

This is because the scaling and feature extraction are not efficient 